In [127]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [128]:
bd_train = pd.read_csv(r'../input/edvancer-project-3-pharma/counterfeit_train.csv')
bd_test = pd.read_csv(r'../input/edvancer-project-3-pharma/counterfeit_test.csv')

In [129]:
test_labels = bd_test['Medicine_ID']

In [130]:
target = bd_train['Counterfeit_Sales']

bd_train.drop('Counterfeit_Sales',axis=1,inplace=True)

bd_train['data']='Train'
bd_test['data']='Test'


In [131]:
all_data = pd.concat([bd_train,bd_test],axis=0)

In [132]:
for col in all_data.columns : 
    
    print(col,' : ',all_data[col].dtype,' : ',all_data[col].isna().sum(),' : ',all_data[col].nunique())

Medicine_ID  :  object  :  0  :  1557
Counterfeit_Weight  :  float64  :  1463  :  415
DistArea_ID  :  object  :  0  :  10
Active_Since  :  int64  :  0  :  9
Medicine_MRP  :  float64  :  0  :  5949
Medicine_Type  :  object  :  0  :  16
SidEffect_Level  :  object  :  0  :  2
Availability_rating  :  float64  :  0  :  7882
Area_Type  :  object  :  0  :  4
Area_City_Type  :  object  :  0  :  3
Area_dist_level  :  object  :  0  :  4
data  :  object  :  0  :  2


In [133]:
all_data['Counterfeit_Weight'].fillna(all_data['Counterfeit_Weight'].median(),inplace=True)

In [134]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

all_data['duplicate_ID'] = le.fit_transform(all_data['Medicine_ID'])

In [135]:
dummy_cols = ['DistArea_ID','Active_Since','Medicine_Type','SidEffect_Level',
             'Area_Type','Area_City_Type','Area_dist_level']

In [136]:
all_data = pd.get_dummies(data=all_data,columns=dummy_cols,drop_first=True)

In [137]:
all_data.drop('Medicine_ID',axis=1,inplace=True)

In [138]:
X = all_data[all_data['data']=='Train']
X.drop('data',axis=1,inplace=True)

x_test = all_data[all_data['data']=='Test']
x_test.drop('data',axis=1,inplace=True)

In [139]:
y = target

In [140]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

X = pd.DataFrame(mms.fit_transform(X),columns = X.columns)
x_test = pd.DataFrame(mms.transform(x_test),columns = x_test.columns)

In [141]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=1)

In [142]:
def report(results, n_top=3): 
    for i in range(1, n_top + 1):
        candidate = np.flatnonzero(results['rank_test_score'] == i)[0]
        print("Model with rank: {0}".format(i))
        print("Mean validation score: {0:.5f} (std: {1:.5f})".format( 
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
        print("Parameters: {0}".format(results['params'][candidate]))
        print("")

In [143]:
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBRegressor

xgb_params = {"n_estimators":[25,50,100,150,200]}

xgb1 = XGBRegressor(subsample = 0.8,
                   colsample_bylevel = 0.8,
                   col_sample_bytree  =0.8,
                   )

In [144]:
xgb = XGBRegressor()

xgb_params = {
              "n_estimators":[25,50,100,150,200],
              'gamma':[0,2,5,8,10],
              'max_depth':[2,3,4,5,6,7,8,9],
              'min_child_weight':[0.5,1,2,5,10],
              'eta':[0.01,0.05,0.1,0.2,0.3,0.5],
              
    
}

'max_delta_step':[0,1,5,10,20,50],
              'subsample':[i/10 for i in range(5,11)], 
              'colsample_bytree':[i/10 for i in range(5,11)],
              'colsample_bylevel':[i/10 for i in range(5,11)],
              'reg_lambda':[i/10 for i in range(0,50)], 
              'reg_alpha':[i/10 for i in range(0,50)]

In [145]:
grid_search = GridSearchCV(xgb,
                          cv =10,
                          param_grid = xgb_params,
                          scoring = 'neg_mean_absolute_error',
                          verbose = False,
                          n_jobs = -1)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
report(grid_search.cv_results_,5)